In [56]:
import tesseract
import os
import re
import numpy as np

In [57]:
t = tesseract.Tesseract()

In [49]:
import tesseract
import os
import re
import numpy as np
import time

def _load_tesseract():
    TESSERACT_SUPPORT_DIR = os.environ["TESSERACT_SUPPORT_DIR"]

    with open(os.path.join(TESSERACT_SUPPORT_DIR,"urdf","lbr_iiwa_14_r820.urdf"),'r') as f:
        robot_urdf = f.read()

    with open(os.path.join(TESSERACT_SUPPORT_DIR,"urdf","lbr_iiwa_14_r820.srdf"),'r') as f:
        robot_srdf = f.read()

    t = tesseract.Tesseract()

    t.init(robot_urdf, robot_srdf, TesseractSupportResourceLocator())

    return t

   

class TesseractSupportResourceLocator(tesseract.ResourceLocator):
    def __init__(self):
        super(TesseractSupportResourceLocator,self).__init__()
        self.TESSERACT_SUPPORT_DIR = os.environ["TESSERACT_SUPPORT_DIR"]

    def locateResource(self, url):
        
        print("url: {}".format(url))
        
        url_match = re.match(r"^package:\/\/tesseract_support\/(.*)$",url)
        if (url_match is None):
            return None
        
        fname = os.path.join(self.TESSERACT_SUPPORT_DIR, os.path.normpath(url_match.group(1)))
        print("fname: {}".format(fname))
        with open(fname,'rb') as f:
            resource_bytes = f.read()

        resource = tesseract.BytesResource(url, resource_bytes)

        return resource
    

In [87]:
import tesseract
import os
import re
import numpy as np
import time
from pkg.utils.utils import *
from pkg.global_config import *

import rospkg
rospack = rospkg.RosPack()
class RosResourceLocator(tesseract.ResourceLocator):
    def __init__(self):
        super(RosResourceLocator,self).__init__()

    def locateResource(self, url):
        fname = None
        if "package://" in url:
            url_split = url[10:].split("/")
            pkg_name = url_split[0]
            pkg_path = rospack.get_path(pkg_name)
            fname = os.path.join(pkg_path, *url_split[1:])
            with open(fname,'rb') as f:
                resource_bytes = f.read()
        else:
            return None

        resource = tesseract.BytesResource(url, resource_bytes)

        return resource

In [100]:
with open(os.path.join(PROJ_DIR, "robots", "custom_robots.urdf"),'r') as f:
    robot_urdf = f.read()

with open(os.path.join(PROJ_DIR, "robots", "custom_robots.srdf"),'r') as f:
    robot_srdf = f.read()

t = tesseract.Tesseract()

t.init(robot_urdf, robot_srdf, RosResourceLocator())
config = tesseract.TrajOptPlannerDefaultConfig(t,"indy0", "indy0_tcp", np.eye(4))
config.params.cnt_tolerance = 1e-5

True

In [102]:
N_test = 4

for ind in range(N_test):
    tcp_pose = np.array([[0,0,1,0],[0,1,0,0],[-1,0,0, 0],[0,0,0,1]],dtype=np.float64)
    tcp_pose[0,3] = 0.5
    tcp_pose[1,3] = -0.2 + ind*.4/(N_test-1)
    tcp_pose[2,3] = 0.5

    waypoint = tesseract.CartesianWaypoint(tcp_pose, "indy0_link0")

    config.target_waypoints.push_back(waypoint)

config.target_waypoints[0].setIsCritical(True)
config.target_waypoints[N_test-1].setIsCritical(True)

planner = tesseract.TrajOptMotionPlanner()


tvec = []
for _ in range(1):
    planner.setConfiguration(config)
    t1 = time.time()
    planner_status_code, planner_response = planner.solve(True)
    t2 = time.time()
    tvec.append(t2-t1)
print(tvec)

assert planner_status_code.value() == 0, "Planning failed"

print(planner_response.joint_trajectory.trajectory)

[4.57387113571167]


AssertionError: 

In [69]:
N_test = 10


t = _load_tesseract()

config = tesseract.TrajOptPlannerDefaultConfig(t,"manipulator", "tool0", np.eye(4))

config.params.cnt_tolerance = 1e-5

for ind in range(N_test):
    tcp_pose = np.array([[0,0,1,0],[0,1,0,0],[-1,0,0, 0],[0,0,0,1]],dtype=np.float64)
    tcp_pose[0,3] = 0.5
    tcp_pose[1,3] = -0.2 + ind*.4/(N_test-1)
    tcp_pose[2,3] = 0.7

    waypoint = tesseract.CartesianWaypoint(tcp_pose, "base_link")

    config.target_waypoints.push_back(waypoint)

config.target_waypoints[0].setIsCritical(True)
config.target_waypoints[N_test-1].setIsCritical(True)

planner = tesseract.TrajOptMotionPlanner()


tvec = []
for _ in range(1):
    planner.setConfiguration(config)
    t1 = time.time()
    planner_status_code, planner_response = planner.solve(True)
    t2 = time.time()
    tvec.append(t2-t1)
print(tvec)

assert planner_status_code.value() == 0, "Planning failed"

assert planner_response.joint_trajectory.trajectory.shape == (N_test,7)
print(planner_response.joint_trajectory.trajectory)

url: package://tesseract_support/urdf/lbr_iiwa_14_r820/visual/base_link.dae
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/visual/base_link.dae
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/collision/base_link.stl
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/collision/base_link.stl
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_1.dae
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_1.dae
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/collision/link_1.stl
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/collision/link_1.stl
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_2.dae
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_2.dae
url: package://tesseract_s

AssertionError: Planning failed

In [7]:
q_s = planner_response.joint_trajectory.trajectory[0,:].copy()
q_e = planner_response.joint_trajectory.trajectory[-1,:].copy()

In [ ]:
print(q_s.astype(np.float16))
print(q_e.astype(np.float16))

In [104]:

t = _load_tesseract()

pci = tesseract.ProblemConstructionInfo(t)

pci.init_info.type = tesseract.InitInfo.STATIONARY
#pci.init_info.data = np.array([0.0,0,0,0,0,0])

pci.basic_info.n_steps = 10
pci.basic_info.manip = "manipulator"
pci.basic_info.start_fixed = True
pci.basic_info.use_time = False

pci.opt_info.max_iter = 10000
pci.opt_info.min_approx_improve = 1e-3
pci.opt_info.min_trust_box_size = 1e-3

pci.kin = pci.getManipulator(pci.basic_info.manip)

start_pos_terminfo = tesseract.JointPosTermInfo()
start_pos_terminfo.name="start"
start_pos_terminfo.term_type = tesseract.TT_CNT
#start_pos_terminfo.coeffs=np.ones(6)
start_pos_terminfo.first_step = 0
start_pos_terminfo.last_step = 1

start_pos_terminfo.targets=q_s*.0 # np.random.rand(7)*0.0

end_pos_terminfo = tesseract.JointPosTermInfo()
end_pos_terminfo.name = "end"
end_pos_terminfo.term_type = tesseract.TT_CNT
#end_pos_terminfo.coeffs=np.ones(6)
end_pos_terminfo.targets=q_e*.05 # np.random.rand(7)*0.1
end_pos_terminfo.first_step = pci.basic_info.n_steps-1
end_pos_terminfo.last_step = pci.basic_info.n_steps-1

joint_vel = tesseract.JointVelTermInfo()
joint_vel.coeffs = np.ones(7)
joint_vel.targets = np.zeros(7)
joint_vel.first_step = 0
joint_vel.last_step = pci.basic_info.n_steps - 1
joint_vel.name = "Joint_vel"
joint_vel.term_type = tesseract.TT_COST

time_terminfo = tesseract.TotalTimeTermInfo()
time_terminfo.coeff = 1
time_terminfo.term_type = tesseract.TT_COST | tesseract.TT_USE_TIME
time_terminfo.name = "time"

collision = tesseract.CollisionTermInfo()
collision.name = "collision"
collision.term_type = tesseract.TT_CNT
collision.continuous = True
collision.first_step = 0
collision.last_step = pci.basic_info.n_steps-2
collision.gap = 1
collision_info = tesseract.createSafetyMarginDataVector(pci.basic_info.n_steps, .0001, 40)
for i in collision_info:
    collision.info.append(i)

pci.cnt_infos.append(start_pos_terminfo)
pci.cnt_infos.append(end_pos_terminfo)
#pci.cost_infos.append(time_terminfo)
pci.cnt_infos.push_back(collision)
pci.cost_infos.push_back(joint_vel)

prob = tesseract.ConstructProblem(pci)
config = tesseract.TrajOptPlannerConfig(prob)

planner = tesseract.TrajOptMotionPlanner()

tvec=[]
planner.setConfiguration(config)
t1 = time.time()
planner_status_code, planner_response = planner.solve(True)
t2 = time.time()
tvec.append(t2-t1)

assert planner_status_code.value() == 0, "Planning failed"

print(planner_response)

print(planner_response.joint_trajectory.trajectory)

print(tvec)

#TODO: test more stuff...

url: package://tesseract_support/urdf/lbr_iiwa_14_r820/visual/base_link.dae
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/visual/base_link.dae
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/collision/base_link.stl
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/collision/base_link.stl
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_1.dae
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_1.dae
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/collision/link_1.stl
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/collision/link_1.stl
url: package://tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_2.dae
fname: /home/tamp/Projects/rnb-planning/ws_ros/devel/share/tesseract_support/urdf/lbr_iiwa_14_r820/visual/link_2.dae
url: package://tesseract_s

In [105]:
planner_response.joint_trajectory.trajectory.shape

(10, 7)

In [36]:

def test_trajopt_motion_planner_default_config():
    t = _load_tesseract()

    config = tesseract.TrajOptPlannerDefaultConfig(t,"manipulator", "tool0", np.eye(4))

    config.params.cnt_tolerance = 1e-5

    for ind in range(10):
        tcp_pose = np.array([[0,0,1,0],[0,1,0,0],[-1,0,0, 0],[0,0,0,1]],dtype=np.float64)
        tcp_pose[0,3] = 0.5
        tcp_pose[1,3] = -0.2 + ind*.04
        tcp_pose[2,3] = 0.7
        
        waypoint = tesseract.CartesianWaypoint(tcp_pose, "base_link")
        
        config.target_waypoints.push_back(waypoint)

    config.target_waypoints[0].setIsCritical(True)
    config.target_waypoints[9].setIsCritical(True)
    
    planner = tesseract.TrajOptMotionPlanner()

    planner.setConfiguration(config)
    planner_status_code, planner_response = planner.solve(True)

    assert planner_status_code.value() == 0, "Planning failed"

    assert planner_response.joint_trajectory.trajectory.shape == (10,7)
    print(planner_response.joint_trajectory.trajectory)

    




def test_trajopt_motion_planner_pci():
    
    t = _load_tesseract()

    pci = tesseract.ProblemConstructionInfo(t)

    pci.init_info.type = tesseract.InitInfo.STATIONARY
    #pci.init_info.data = np.array([0.0,0,0,0,0,0])

    pci.basic_info.n_steps = 10
    pci.basic_info.manip = "manipulator"
    pci.basic_info.start_fixed = True
    pci.basic_info.use_time = False

    pci.opt_info.max_iter = 10000
    pci.opt_info.min_approx_improve = 1e-3
    pci.opt_info.min_trust_box_size = 1e-3

    pci.kin = pci.getManipulator(pci.basic_info.manip)

    start_pos_terminfo = tesseract.JointPosTermInfo()
    start_pos_terminfo.name="start"
    start_pos_terminfo.term_type = tesseract.TT_CNT
    #start_pos_terminfo.coeffs=np.ones(6)
    start_pos_terminfo.first_step = 0
    start_pos_terminfo.last_step = 1

    start_pos_terminfo.targets=np.random.rand(7)*0.0

    end_pos_terminfo = tesseract.JointPosTermInfo()
    end_pos_terminfo.name = "end"
    end_pos_terminfo.term_type = tesseract.TT_CNT
    #end_pos_terminfo.coeffs=np.ones(6)
    end_pos_terminfo.targets=np.random.rand(7)*0.1
    end_pos_terminfo.first_step = pci.basic_info.n_steps-1
    end_pos_terminfo.last_step = pci.basic_info.n_steps-1

    joint_vel = tesseract.JointVelTermInfo()
    joint_vel.coeffs = np.ones(7)
    joint_vel.targets = np.zeros(7)
    joint_vel.first_step = 0
    joint_vel.last_step = pci.basic_info.n_steps - 1
    joint_vel.name = "Joint_vel"
    joint_vel.term_type = tesseract.TT_COST

    time_terminfo = tesseract.TotalTimeTermInfo()
    time_terminfo.coeff = 1
    time_terminfo.term_type = tesseract.TT_COST | tesseract.TT_USE_TIME
    time_terminfo.name = "time"

    collision = tesseract.CollisionTermInfo()
    collision.name = "collision"
    collision.term_type = tesseract.TT_CNT
    collision.continuous = True
    collision.first_step = 0
    collision.last_step = pci.basic_info.n_steps-2
    collision.gap = 1
    collision_info = tesseract.createSafetyMarginDataVector(pci.basic_info.n_steps, .0001, 40)
    for i in collision_info:
        collision.info.append(i)

    pci.cnt_infos.append(start_pos_terminfo)
    pci.cnt_infos.append(end_pos_terminfo)
    #pci.cost_infos.append(time_terminfo)
    pci.cnt_infos.push_back(collision)
    pci.cost_infos.push_back(joint_vel)

    prob = tesseract.ConstructProblem(pci)
    config = tesseract.TrajOptPlannerConfig(prob)

    planner = tesseract.TrajOptMotionPlanner()

    planner.setConfiguration(config)
    planner_status_code, planner_response = planner.solve(True)
                
    assert planner_status_code.value() == 0, "Planning failed"

    print(planner_response)
    
    print(planner_response.joint_trajectory.trajectory)
        

    #TODO: test more stuff...
    



In [38]:
test_trajopt_motion_planner_pci()

<tesseract.tesseract_python.PlannerResponse; proxy of <Swig Object of type 'tesseract_motion_planners::PlannerResponse *' at 0x7fb50e200d80> >
[[-3.34734137e-24  1.44724028e-23 -5.37123012e-24 -7.01762051e-25
   1.79814170e-23 -2.81007853e-24 -2.48093764e-23]
 [ 1.98545287e-21  4.08846925e-21  3.18627005e-21  5.27217566e-22
   1.89462866e-21  1.88915713e-21  2.26504891e-21]
 [ 6.15002629e-03  1.23178954e-02  9.86845488e-03  1.28931904e-03
   5.87071125e-03  5.16295531e-03  6.67413993e-03]
 [ 1.23000526e-02  2.46357907e-02  1.97369098e-02  2.57863808e-03
   1.17414225e-02  1.03259106e-02  1.33482799e-02]
 [ 1.84500789e-02  3.69536861e-02  2.96053646e-02  3.86795712e-03
   1.76121337e-02  1.54888659e-02  2.00224198e-02]
 [ 2.46001051e-02  4.92715815e-02  3.94738195e-02  5.15727616e-03
   2.34828450e-02  2.06518212e-02  2.66965597e-02]
 [ 3.07501314e-02  6.15894768e-02  4.93422744e-02  6.44659520e-03
   2.93535562e-02  2.58147765e-02  3.33706996e-02]
 [ 3.69001577e-02  7.39073722e-02  5.9